In [1]:
# Import necessary libraries
import pandas as pd
from ast import literal_eval
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import warnings
import ast
import seaborn as sns
import itertools
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from scipy.stats import anderson_ksamp, kruskal, ks_2samp



In [2]:
test = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_test_with_embeddings_extracted.pkl')
train = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_train_with_embeddings_extracted.pkl')
valid = pd.read_pickle('../data/embeddings_chexpert/CNNs/chexpert_on_chexpert_valid_with_embeddings_extracted.pkl')
test.columns

Index(['gender', 'race', 'age', 'insurance', 'Atelectasis', 'Cardiomegaly',
       'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture',
       'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion',
       'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices',
       'embeddings'],
      dtype='object')

In [3]:
col = ['gender', 'race', 'age', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices', 'embeddings']
test = test[col]
train = train[col]
valid = valid[col]


In [4]:
col = ['sex', 'race', 'age', 'Atelectasis',
       'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum',
       'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding',
       'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax',
       'Support Devices', 'embeddings']
test.columns = col
train.columns = col
valid.columns = col

In [5]:
initial_size = test.shape[0] 
test = test[test['embeddings'].apply(type) == list]
final_size = test.shape[0] 
print(f'Number of test removed rows = {initial_size - final_size}')

initial_size = train.shape[0] 
train = train[train['embeddings'].apply(type) == list]
final_size = train.shape[0] 
print(f'Number of train removed rows = {initial_size - final_size}')

initial_size = valid.shape[0]
valid = valid[valid['embeddings'].apply(type) == list]
final_size = valid.shape[0]
print(f'Number of valid removed rows = {initial_size - final_size}')

Number of test removed rows = 51
Number of train removed rows = 67
Number of valid removed rows = 3


## Proportion distribution

In [6]:
import pandas as pd

# your setup
diseases_of_interest = ['Cardiomegaly', 'Lung Opacity', 'Edema', 'Pleural Effusion']
# df = train[train[diseases_of_interest].any(axis=1)].copy()

df = train
sex_map  = {0: 'Male',   1: 'Female'}
race_map = {0: 'White',  1: 'Asian', 2: 'Black'}
age_map  = {0: 'Young',  1: 'Old'} 

df['Sex']  = df['sex'].map(sex_map)
df['Race'] = df['race'].map(race_map)
df['Age']  = df['age'].map(age_map)

# loop and print
for disease in diseases_of_interest:
    print(f"\n=== {disease} ===")
    for col in ['Sex', 'Race', 'Age']:
        ct = pd.crosstab(df[col], df[disease])\
               .rename(columns={0: '-', 1: '+'})\
               .astype(int)
        print(f"\n{col} distribution:\n{ct}\n")



=== Cardiomegaly ===

Sex distribution:
Cardiomegaly      -     +
Sex                      
Female        25055  3143
Male          33931  5067


Race distribution:
Cardiomegaly      -     +
Race                     
Asian          8594  1250
Black          3901   898
White         46491  6062


Age distribution:
Cardiomegaly      -     +
Age                      
Old           21069  3860
Young         37917  4350


=== Lung Opacity ===

Sex distribution:
Lung Opacity      -      +
Sex                       
Female        14245  13953
Male          19681  19317


Race distribution:
Lung Opacity      -      +
Race                      
Asian          5018   4826
Black          2508   2291
White         26400  26153


Age distribution:
Lung Opacity      -      +
Age                       
Old           11590  13339
Young         22336  19931


=== Edema ===

Sex distribution:
Edema       -     +
Sex                
Female  21422  6776
Male    29780  9218


Race distribution:
Edema     

In [7]:
train.shape

(67196, 21)

In [8]:
records = []
for disease in diseases_of_interest:
    for col in ['Sex','Race','Age']:
        for cat, group in df.groupby(col):
            neg = round(int(((group[disease] == 0)).sum()) / 67196 * 100, 1)
            pos = round(int(((group[disease] == 1)).sum()) / 67196 * 100, 1)
            percent_pos = round(pos / (pos + neg) * 100, 1)
            records.append({
                'Disease':  disease,
                'Variable': col,
                'Category': cat,
                'Negative': neg,
                'Positive': pos,
                'Percent Positive': percent_pos
            })
summary_df = pd.DataFrame(records)
summary_df

,Disease,Variable,Category,Negative,Positive,Percent Positive
0,Cardiomegaly,Sex,Female,37.3,4.7,11.2
1,Cardiomegaly,Sex,Male,50.5,7.5,12.9
2,Cardiomegaly,Race,Asian,12.8,1.9,12.9
3,Cardiomegaly,Race,Black,5.8,1.3,18.3
4,Cardiomegaly,Race,White,69.2,9.0,11.5
5,Cardiomegaly,Age,Old,31.4,5.7,15.4
6,Cardiomegaly,Age,Young,56.4,6.5,10.3
7,Lung Opacity,Sex,Female,21.2,20.8,49.5
8,Lung Opacity,Sex,Male,29.3,28.7,49.5
9,Lung Opacity,Race,Asian,7.5,7.2,49.0


# Predict subgroups from embeddings

In [9]:
# Extract embedding arrays and standardize
def prepare_X(df):
    X = np.stack(df['embeddings'].values)
    return X

X_train = prepare_X(train)
X_valid = prepare_X(valid)
X_test  = prepare_X(test)

scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled  = scaler.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression

y_train = train['sex'].values
y_valid = valid['sex'].values
y_test  = test['sex'].values


clf = LogisticRegression()

clf.fit(X_train_scaled, y_train)
y_pred_test = clf.predict(X_test_scaled)
y_proba_test = clf.predict_proba(X_test_scaled)[:, 1]

acc_test = accuracy_score(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, y_proba_test)

print(f"Sex Test Accuracy: {acc_test:.4f}")
print(f"Sex Test AUC:      {auc_test:.4f}")

Sex Test Accuracy: 0.8567
Sex Test AUC:      0.9315


/home/cmottez/miniconda3/envs/mitigate-bias/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
y_train = train['age'].values
y_valid = valid['age'].values
y_test  = test['age'].values



clf = LogisticRegression()

clf.fit(X_train_scaled, y_train)
y_pred_test = clf.predict(X_test_scaled)
y_proba_test = clf.predict_proba(X_test_scaled)[:, 1]

acc_test = accuracy_score(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, y_proba_test)

print(f"Age Test Accuracy: {acc_test:.4f}")
print(f"Age Test AUC:      {auc_test:.4f}")

Age Test Accuracy: 0.7521
Age Test AUC:      0.8162


/home/cmottez/miniconda3/envs/mitigate-bias/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
y_train = train['race'].values
y_valid = valid['race'].values
y_test  = test['race'].values


clf = LogisticRegression()


clf.fit(X_train_scaled, y_train)


y_pred_test  = clf.predict(X_test_scaled)
y_proba_test = clf.predict_proba(X_test_scaled)
acc_test = accuracy_score(y_test, y_pred_test)

# multiclass AUC: specify multi_class and average
#    - multi_class='ovr' does one-vs-rest
#    - average='macro' unweights each class equally
auc_test = roc_auc_score(
    y_test,
    y_proba_test,
    multi_class='ovr',
    average='macro'
)

print(f"Race Test Accuracy: {acc_test:.4f}")
print(f"Race Test AUC (OvR, macro‐avg): {auc_test:.4f}")


Race Test Accuracy: 0.7938
Race Test AUC (OvR, macro‐avg): 0.7696


/home/cmottez/miniconda3/envs/mitigate-bias/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
